In [28]:
import pandas as pd
import requests as req
import numpy as np
import re

In [3]:
# objectives: get longitude, latitude, zipcode

In [14]:
res = req.get('https://nominatim.openstreetmap.org/search?street=181 Front Street&state=New York&city=Brooklyn&addressdetails=1&format=json')
res.json()

[{'place_id': 238740394,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 684690680,
  'boundingbox': ['40.7024618', '40.7026248', '-73.9863777', '-73.9855975'],
  'lat': '40.70252645',
  'lon': '-73.98598783528503',
  'display_name': '181 Front Street, Front Street, Brooklyn, New York, 11201, United States',
  'class': 'building',
  'type': 'yes',
  'importance': 0.611,
  'address': {'building': '181 Front Street',
   'road': 'Front Street',
   'suburb': 'Brooklyn',
   'city': 'New York',
   'state': 'New York',
   'postcode': '11201',
   'country': 'United States',
   'country_code': 'us'}}]

In [34]:
fakeaddy = '3676 Richmond 1-FLOOR Avenue 1ST-ST-FLoor'
re.sub('([a-zA-Z0-9]{1,}-)*FLOOR', '', str(fakeaddy).strip(), flags=re.IGNORECASE).strip()

'3676 Richmond  Avenue'

In [36]:
# bronx 
def get_loc_data(street, boro):
    state = "New York"
    city = str(boro).strip()
    # address = str(street).strip()
    address = re.sub('([a-zA-Z0-9]{1,}-)*FLOOR', '', str(street).strip(), flags=re.IGNORECASE).strip()
    query = f"https://nominatim.openstreetmap.org/search?street={address}&state={state}&city={city}&addressdetails=1&limit=1&format=json"
    result = req.get(query).json()
    
    if len(result) < 1:
        return np.NaN, np.NaN, np.NaN

    data = result[0]
    # addy_split = data['display_name'].split(',')
    return [float(data['lon'].strip()), float(data['lat'].strip()), data['address']['postcode'].strip()]

In [37]:
stat_csv = '../Preprocessing/ImputedSets/staten_imputed.csv'
stat_df = pd.DataFrame(pd.read_csv(stat_csv))
stat_df.head()

,Borough,Area,Address,Num_Beds,Num_Baths,Sq-Ft,Rent
0,staten-island,New Dorp Beach,48C Winham Avenue,0.5,1.0,513.32,1199.0
1,staten-island,Eltingville,3676 Richmond Avenue 1-FLOOR,3.0,1.0,1500.00,2400.0
2,staten-island,Saint George,36 Hamilton Avenue,2.0,1.0,900.00,2150.0
3,staten-island,Saint George,36 Hamilton Avenue,1.0,1.0,750.00,1600.0
4,staten-island,Saint George,60 Hamilton Avenue,2.0,1.0,833.22,1950.0


In [43]:
stat_df[['Long', 'Lat', 'ZipCode']] = np.nan

In [44]:
stat_df.head()


,Borough,Area,Address,Num_Beds,Num_Baths,Sq-Ft,Rent,Long,Lat,ZipCode
0,staten-island,New Dorp Beach,48C Winham Avenue,0.5,1.0,513.32,1199.0,NaN,NaN,NaN
1,staten-island,Eltingville,3676 Richmond Avenue 1-FLOOR,3.0,1.0,1500.00,2400.0,NaN,NaN,NaN
2,staten-island,Saint George,36 Hamilton Avenue,2.0,1.0,900.00,2150.0,NaN,NaN,NaN
3,staten-island,Saint George,36 Hamilton Avenue,1.0,1.0,750.00,1600.0,NaN,NaN,NaN
4,staten-island,Saint George,60 Hamilton Avenue,2.0,1.0,833.22,1950.0,NaN,NaN,NaN


In [47]:
stat_df.iloc[0]

Borough          staten-island
Area            New Dorp Beach
Address      48C Winham Avenue
Num_Beds                   0.5
Num_Baths                  1.0
Sq-Ft                   513.32
Rent                    1199.0
Long                       NaN
Lat                        NaN
ZipCode                    NaN
Name: 0, dtype: object

In [46]:
# stat_df['Long'], stat_df['Lat'], stat_df['ZipCode'] = stat_df.apply(lambda row: get_loc_data(row['Address'], 'Staten-Island'), axis=1)


for idx, row in stat_df.iterrows():
    data = get_loc_data(row['Address'], 'Staten-Island')
    stat_df.iloc[idx:idx+1, 'Long'] = data[0]
    stat_df.iloc[idx:idx+1, 'Lat'] = data[1]
    stat_df.iloc[idx:idx+1, 'ZipCode'] = data[2]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices